In [39]:
import pandas as pd
from pymongo import MongoClient
import numpy as np
import scipy.sparse as  sps
import scipy.io as io

In [33]:
host = 'localhost'
port = '27017'
db = 'alizeh'
client = MongoClient('mongodb://localhost:27017/alizeh')
dbCursor = client['alizeh']
cursor = dbCursor['user_activity'].find({})
df =  pd.DataFrame(list(cursor))
df.shape


# db = client.database_name
# collection = db.collection_name
# data = pd.DataFrame(list(collection.find()))

(9254, 7)

In [80]:
pd.isna(df['rating'])
# df[df['visited_post'][] != 'NaN']

In [4]:
extracted_df = df[['blogpost_id', 'rating', 'user_id']].copy()
extracted_df.dropna(axis=0, inplace=True)

In [5]:
extracted_df.set_index(['user_id', 'blogpost_id'], inplace=True)
extracted_df
# extracted_df[extracted_df.index.labels[1] < 0]

rating
user_id                  blogpost_id         
5acb783dab23ba134d0580c8 140b4bf94d9f     2.0
                         33a063e8b147     5.0
                         f74463ed54ee     5.0
                         aa8325c6145f     1.0
                         bd4b418f3cfb     5.0
                         db803f9fd1b7     4.0
                         99707a3a6126     2.0
                         8c1bca975c4e     3.0
                         7ec85cb35756     1.0
                         dca93fec9e03     4.0
                         1ad7e58a7127     1.0
                         1d3b73caad02     2.0
                         2f2bc9aa3c54     4.0
                         d7d5a528394c     4.0
                         f2f51a29c572     5.0
                         13079bc7aaef     2.0
                         31c052327b8b     5.0
                         29eacd3735e1     2.0
5acb783dab23ba134d0580c9 2854c58ffb8c     2.0
                         33d2a28534f2     4.0
                         b51cd7955349     2.0
                         855a5e6424db     3.0
                         8f785853b53      5.0
                         e714b699c38b     2.0
                         88a52f7bab6f     1.0
                         797ef03cdef2     2.0
                         89876fd68455     3.0
                         383179f9a519     4.0
                         32244b58acec     1.0
                         2334638d4a65     3.0
...                                       ...
5acb783eab23ba134d05829c 2e4e5a3f677d     3.0
                         1fc50bd1267b     4.0
                         1b39d1581d86     2.0
                         7fb404b60a13     4.0
                         75ca88917582     3.0
                         c41a2d9c48bb     2.0
                         18348eecf270     5.0
                         709ebcb898dc     2.0
                         5e7f2ec4be94     3.0
                         cbab161000f7     2.0
                         519e94d19dba     1.0
5acb783eab23ba134d0582a4 71698d36bb96     4.0
                         f74a99087f26     2.0
                         20b661100819     4.0
                         209576606c54     1.0
                         995d23687ad      2.0
                         116fe6c9ae8b     4.0
                         2376317be109     3.0
                         b325a67f2c3      1.0
                         9acf2a1944f0     1.0
                         143d954785cb     2.0
                         bba9ed602c32     4.0
                         ba69df05804d     4.0
                         cda79679bbe3     2.0
                         1e57974c7d4a     1.0
                         7a4ed8994a67     4.0
5acb783eab23ba134d0582a9 344a8b67fba1     1.0
                         1871fb51cc21     2.0
                         2334638d4a65     3.0
                         85258e1bc87      3.0

[9253 rows x 1 columns]

In [12]:
# spsMat = sps.coo_matrix((extracted_df.rating,((extracted_df.index.labels[0], extracted_df.index.labels[1]))))
# spsMat.todense()
# io.mmwrite('test123',spsMat)

In [34]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(extracted_df,test_size=0.25)
train.shape

(6939, 1)

In [17]:
spsTrain = sps.coo_matrix((train.rating,((train.index.labels[0], train.index.labels[1]))))
# spsMat.todense()
# io.mmwrite('test123',spsMat)

In [29]:
from lightfm import LightFM
model = LightFM(loss='warp')
%time model.fit(spsTrain, epochs=15, num_threads=3)


CPU times: user 235 ms, sys: 0 ns, total: 235 ms
Wall time: 96 ms


In [37]:
train_blog_ids, train_user_id = spsTrain.shape # blog X user

In [40]:
# for user_id in user_ids:
#         known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

scores = model.predict(train_blog_ids, np.arange(train_user_id))

ValueError: The user feature matrix specifies more features than there are estimated feature embeddings: 478 vs 479.